Demonstrate functions in Scan for setting and reading tones.  Readback can give all samples or the mean.

Note that after setting new tones, the first samples are "left over" from previous reads.  The paramter "iBegin" can be used to remove these samples from the returned values or average.

In [ ]:
import importlib
from mkids import TopSoc
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import trange, tqdm
import pandas as pd
import Scan
soc = TopSoc()

In [ ]:
scan = Scan.Scan(soc)
fMixer = soc.fsIn/2/2
scan.soc.set_mixer(fMixer)

In [ ]:
# Setup tones that are in nearby channels, to fully test multi-tone readout
f0 = fMixer + 1.4
nTones = 10
freqs = f0 + 0.1 + np.arange(nTones)*(1.2*soc.fcOut+0.01)
# Move one tone away from the others
freqs[0] += 34.56
freqs[3] -= 123.456

inCh, dds_freq, pfb_freq, ch = soc.pfb_in.freq2ch(freqs)   
outCh, outOffset = soc.outFreq2chOffset(freqs)
inCh, inOffset = soc.inFreq2chOffset(freqs)
data = np.transpose(np.array([freqs, outCh, outOffset, inCh, inOffset]))
pd.DataFrame(data, columns=["freq","outCh","outOffset", "inCh", "inOffset"])

In [ ]:
amps = 0.9*np.ones(len(freqs))/len(freqs)
fis = 1+(0.1*np.arange(len(freqs)))
scan.setTones(freqs, amps, fis)
decimation = 2
scan.prepRead(decimation)

nt = 1
nsamp = 2000
truncate = 1000
x = scan.readAndUnpack(nt=nt, nsamp=nsamp,unpackVerbose=True)

In [ ]:
it = 0
for iTone in range(len(x[it])):
    plt.plot(np.abs(x[it][iTone]), label="f=%f"%(freqs[iTone]))
plt.legend()
plt.ylabel("Amplitude (ADUs)")

In [ ]:
it = 0
for iTone in range(len(x[it])):
    plt.plot(np.angle(x[it][iTone]), label="f=%f"%(freqs[iTone]))
plt.legend()
plt.ylabel("Phase (Radians)")

In [ ]:
subtractInputPhase=False

# Setup tones that are in nearby channels, to fully test multi-tone readout
f0 = fMixer + 1.4
nTones = 10
freqs = f0 + 0.1 + np.arange(nTones)*(1.2*soc.fcOut+0.01)
# Move one tone away from the others
freqs[0] += 34.56
freqs[3] -= 123.456
amps = 0.9*np.ones(len(freqs))/len(freqs)
fis = 1+(0.1*np.arange(len(freqs)))
scan.setTones(freqs, amps, fis)
decimation = 2
scan.prepRead(decimation)
nt = 2
nsamp = 20000
truncate = 1000
x = scan.readAndUnpack(nt=nt, nsamp=nsamp, subtractInputPhase=subtractInputPhase)

In [ ]:
iTone = 0
for it in range(nt):
    plt.plot(np.angle(x[it][iTone]), label="it=%d iTone=%d"%(it,iTone), alpha=0.2)
plt.legend()

In [ ]:
#test average=True in unpack
verbose = True
average=True
iBegin=0
xa = scan.unpack(verbose, average, subtractInputPhase, iBegin)

In [ ]:
xa.shape

In [ ]:
iTone = 3
for it in range(nt):
    plt.plot(np.angle(x[it][iTone]), label="it=%d iTone=%d"%(it,iTone), alpha=0.2)
    plt.axhline(np.angle(xa[iTone]), c='r')
plt.legend()
plt.ylabel("Phase [Radians]")
plt.title("red line is mean of both reads")

In [ ]:
soc.adcs